<h2><center>Final Project: Where it Pays to Attend College</center></h2>

<h3><center>By Ansh Vanam and Rishi Mullangi<center></h3>

<h2>Introduction</h2>

For our final project, we have decided to work with a dataset that contains salaries of various colleges based on major, type of school (private, public), and region. The dataset we chose has three separate CSV files, "salaries-by-college-type.csv", "salaries-by-region.csv", and "degrees-that-pay-back.csv". These CSV files contain data from graduating up to their mid-career, including percentiles ranging from 10-90. 

Using these CSV files, we plan to look into how these variables affect the salaries of students, and how these factors can affect choices of students looking to go into college. Our first step is to download the CSV files from this Kaggle dataset, created by the Wall Street Journal:

https://www.kaggle.com/datasets/wsj/college-salaries/code?resource=download&select=salaries-by-college-type.csv

From here, we can download the three CSV files and import them into our code editor. Then, we use pandas to create dataframes out of the CSV files. 

Note: Ensure you have the following python libraries installed on your system and the CSV files downloaded to run the tutorial.

Pandas documentation - https://pandas.pydata.org/docs/ 

Seaborn documentation - https://seaborn.pydata.org

Numpy documentation - https://numpy.org/doc/ 

Matplotlib documentation - https://matplotlib.org/stable/index.html 

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as stm

The first table we're looking at is salaries-by-college-type, which details the names of various colleges as well as their School Type. In the next python block we will work on cleaning the data and making it more useable. 

In [ ]:
df_by_colleges = pd.read_csv('salaries-by-college-type.csv')
#Getting rid of all of the NaN values and replacing them with 0 to make calculations doable.
#df1 = df1.fillna(0)
#Might want better names here, I couldn't think of anything for now. 
df_by_colleges = df_by_colleges.rename(columns = {'School Name':'School_Name','School Type':'School_Type','Starting Median Salary':'Starting_Median_Salary', 'Mid-Career Median Salary':'Mid-Career_Median_Salary', 'Mid-Career 10th Percentile Salary':'Mid-Career_10th_Percentile_Salary',
'Mid-Career 25th Percentile Salary':'Mid-Career_25th_Percentile_Salary', 'Mid-Career 75th Percentile Salary':'Mid-Career_75th_Percentile_Salary', 'Mid-Career 90th Percentile Salary':'Mid-Career_90th_Percentile_Salary'})
df_by_colleges.head()

Now that we've cleaned up our first table by converting all the NaN values to zeroes, we can move on to the next table, 'degrees-that-pay-back.csv', which details undergraduate major's median starting salary, mid-career median salary, and other useful information that we can use later on in the tutorial. 

In [ ]:
df_by_major = pd.read_csv('degrees-that-pay-back.csv')
#Getting rid of all of the NaN values and replacing them with 0 to make calculations doable.
#df_by_major = df_by_major.fillna(0)
df_by_major.head()

Finally, we can look at the last CSV file, 'salaries-by-region.csv', which details college graduate salaries by region and college.

In [ ]:
df_by_region = pd.read_csv('salaries-by-region.csv')
#Getting rid of all of the NaN values and replacing them with 0 to make calculations doable.
#df_by_region = df_by_region.fillna(0)
df_by_region.head()

<h2>Data Management/Representation</h2>
Next, we will represent various data to try and understand the tables more. 

In [ ]:
df_by_region['Region'].nunique()

In [ ]:
len(df_by_region['School Name'].unique())

In [ ]:
df_by_region[df_by_region['School Name'] == 'Massachusetts Institute of Technology (MIT)']

In [ ]:
df_by_colleges[df_by_colleges['School_Name'] == 'Massachusetts Institute of Technology (MIT)']

In [ ]:
df_by_region